# This graph learning treats every graph (CP) to be an episodic memory of the agent.

## Turn the raw data into RDF-data

I also did ontology engineering, which is saved at [`ontology-humemai.ttl`](./ontology-humemai.ttl)


In [1]:
import json

with open('./raw-data.json') as f:
    data = json.load(f)

In [2]:
from pprint import pprint

pprint(data[1])

{'actionHuman': [[{'content': 'Stand still in <location>', 'type': 'action'},
                  {'content': '<Right> side of field', 'type': 'location'}],
                 [],
                 []],
 'actionRobot': [[],
                 [{'content': 'Break <object> in <location>', 'type': 'action'},
                  {'content': 'Large rock', 'type': 'object'},
                  {'content': 'Top of rock pile', 'type': 'location'}],
                 []],
 'cp_name': 'stand still break big rock',
 'cp_num': 1,
 'participant': 4071,
 'remaining_rocks': 23,
 'remaining_time': 3000,
 'round_num': 1,
 'situation': [[{'content': 'Large rock', 'type': 'object'},
                {'content': 'Top of rock pile', 'type': 'location'}],
               [{'content': 'Large rock', 'type': 'object'},
                {'content': '<Left> side of rock pile', 'type': 'location'}],
               [{'content': '<Right> side of field', 'type': 'location'},
                {'content': 'Human', 'type': 'actor'}]]

In [14]:
import logging
from rdflib import URIRef, Namespace
from humemai import MemorySystem

# Turn off debug logging
logging.getLogger("humemai").setLevel(logging.WARNING)

# Define the custom namespace for humemai ontology
humemai = Namespace("https://humem.ai/ontology/")
co_learning = Namespace("https://co_learning/")


memory_system = MemorySystem(verbose_repr=False)
memory_system.memory.graph.bind("co_learning", co_learning)

memory_system.memory.add_long_term_memory(
    memory_type="episodic",
    triples=[
        (
            URIRef("https://co_learning/entity/participant/P4069"),
            URIRef("https://co_learning/relation/hasSituation"),
            URIRef("https://co_learning/entity/situation/S00"),
        ),
    ],
    location="USAR",
    time="2024-05-03T13:41:23",
    event="CP000",
)
memory_system.memory.add_long_term_memory(
    memory_type="episodic",
    triples=[
        (
            URIRef("https://co_learning/entity/participant/P4069"),
            URIRef("https://co_learning/relation/hasSituation"),
            URIRef("https://co_learning/entity/situation/S00"),
        ),
    ],
    location="USAR",
    time="2024-05-03T13:41:23",
    event="CP001",
)

In [15]:
memory_system.memory

[P4069, hasSituation, S00, {'memoryID': '0', 'location': 'USAR', 'time': '2024-05-03T13:41:23', 'event': 'CP000', 'recalled': '0'}]
[P4069, hasSituation, S00, {'memoryID': '1', 'location': 'USAR', 'time': '2024-05-03T13:41:23', 'event': 'CP001', 'recalled': '0'}]

In [16]:
memory_system.memory.get_memories(event="CP000")

[P4069, hasSituation, S00, {'memoryID': '0', 'location': 'USAR', 'time': '2024-05-03T13:41:23', 'event': 'CP000', 'recalled': '0'}]

In [22]:
triples = [
    (URIRef("https://humem.ai/ontology/hasName"), humemai.hasName, None),
]

from rdflib import Graph, Namespace

humemai = Namespace("https://humem.ai/ontology/")
co_learning = Namespace("https://humem.ai/ontology/co-learning/")

# Create a new RDF graph
g = Graph()

# Bind the namespaces to prefixes in the RDF graph
g.bind("humemai", humemai)
g.bind("co_learning", co_learning)

# Now you can use these namespaces for your triples or qualifiers
subject = humemai.Alice
predicate = co_learning.learns
object_ = humemai.AI

# Adding triples to the graph
g.add((subject, predicate, object_))

# Example to view what’s been added
for s, p, o in g:
    print(s, p, o)

    


https://humem.ai/ontology/Alice https://humem.ai/ontology/co-learning/learns https://humem.ai/ontology/AI


In [21]:
g

<Graph identifier=N42a3b594ab1544b887e1857c07c92968 (<class 'rdflib.graph.Graph'>)>

In [19]:
co_learning.p4069

rdflib.term.URIRef('https://humem.ai/ontology/co-learning/p4069')

In [12]:
co_learning

Namespace('https://humem.ai/ontology/co-learning/')

In [13]:
co_learning.foo

rdflib.term.URIRef('https://humem.ai/ontology/co-learning/foo')

In [16]:
URIRef(co_learning.foo)

rdflib.term.URIRef('https://humem.ai/ontology/co-learning/foo')

In [27]:
humemai = Namespace("https://humem.ai/ontology/")
URIRef("https://humem.ai/ontology/foo/bar")
humemai.foo.bar

rdflib.term.URIRef('https://humem.ai/ontology/foo/bar')

AttributeError: 'URIRef' object has no attribute 'bar'